## Starting the server
%pip install -U langgraph-cli

```langgraph build -t my-image```

```docker compose up```


## Making requests to server

In [1]:
%pip install -U langchain
%pip install -U langgraph_sdk

In [2]:
from langgraph_sdk import get_client

url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [3]:
# Crate thread
thread = await client.threads.create()
runs = await client.runs.list(thread_id=thread['thread_id'])
print(runs)
thread

[]


{'thread_id': 'a85056be-e2c8-4b5d-9a9f-811ae88315f8',
 'created_at': '2025-01-12T15:33:43.046398+00:00',
 'updated_at': '2025-01-12T15:33:43.046398+00:00',
 'metadata': {},
 'status': 'idle',
 'config': {},
 'values': None,
 'interrupts': {}}

In [4]:
from langchain_core.messages import HumanMessage
user_input = "What is 2+3? Then add 60. Then add 90. Then add 160. Then add 450."
config = {"configurable":{"user_id":"Test"}}
graph_name = "agent"
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages":[HumanMessage(content=user_input)]},config=config)

In [6]:
user_input = "What is 2+3? Then add 60. Then add 90. Then add 160. Then add 450."
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)

565155315765The result of the given operations is 765.

In [7]:
from langchain_core.messages import convert_to_messages
thread_state = await client.threads.get_state(thread['thread_id'])
for m in convert_to_messages(thread_state['values']['messages']):
    m.pretty_print()

================================ Human Message =================================

What is 2+3? Then add 60. Then add 90. Then add 160. Then add 450.
================================== Ai Message ==================================
Tool Calls:
  add (call_zxs7)
 Call ID: call_zxs7
  Args:
    a: 2
    b: 3
  add (call_nwrv)
 Call ID: call_nwrv
  Args:
    a: 5
    b: 60
  add (call_b7nm)
 Call ID: call_b7nm
  Args:
    a: 65
    b: 90
  add (call_dphv)
 Call ID: call_dphv
  Args:
    a: 155
    b: 160
  add (call_7h1g)
 Call ID: call_7h1g
  Args:
    a: 315
    b: 450
================================= Tool Message =================================
Name: add

5
================================= Tool Message =================================
Name: add

65
================================= Tool Message =================================
Name: add

155
================================= Tool Message =================================
Name: add

315
================================= Tool Message